# Fluvial flood risk

This example shows a workflow to derive fluvial flood risk using the **Wflow**, **SFINCS** and **Delft-FIAT** models. The starting point is a user defined region and data catalog. Wflow simulated discharge is translated into hydrographs for different return periods and used to simulate the flood hazard maps. The hazard maps are combined with exposure and impact data to derive risk.

In [ ]:
# Import packages
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog
from hydroflows.methods import discharge, fiat, sfincs, wflow
from hydroflows.utils.example_data import fetch_data

logger = setuplog(level="INFO")

In [ ]:
# Define case name and root directory
name = "fluvial_risk"
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory
pwd_rel = "../../"  # relative path from the case directory to the current file

## Workflow inputs

The example requires the following inputs which are provided via a configuration file:
- a user defined region that can be used to delineate the SFINCS model domain
- a data catalog file describing all input datasets. Here we fetch some test datasets for a region in Northern Italy.  
- HydroMT configuration files for all three models
- model executables (docker is also possible for SFINCS and Wflow)

In [ ]:
# Fetch the global build data
cache_dir = fetch_data(data="global-data")

In [ ]:
# Setup the configuration
config = WorkflowConfig(
    # general settings
    region=Path(pwd_rel, "data/build/region.geojson"),
    catalog_path=Path(cache_dir, "data_catalog.yml"),
    plot_fig=True,
    start_date="2014-01-01",
    end_date="2021-12-31",
    # sfincs settings
    hydromt_sfincs_config=Path(pwd_rel, "hydromt_config/sfincs_config.yml"),
    sfincs_exe=Path(pwd_rel, "bin/sfincs_v2.1.1/sfincs.exe"),
    depth_min=0.05,  # minimum depth for inundation map
    # fiat settings
    hydromt_fiat_config=Path(pwd_rel, "hydromt_config/fiat_config.yml"),
    fiat_exe=Path(pwd_rel, "bin/fiat_v0.2.0/fiat.exe"),
    # wflow settings
    hydromt_wflow_config=Path(pwd_rel, "hydromt_config/wflow_config.yml"),
    wflow_exe=Path(pwd_rel, "bin/wflow_v0.8.1/bin/wflow_cli.exe"),
    # design event settings
    rps=[2, 5, 10, 50, 100],
)


## Create the workflow

In [ ]:
# create and empty workflow
wf = Workflow(name=name, config=config, root=case_root)

### Build models

In this section we build a model cascade and make sure these are configured correctly for offline coupling, i.e. Wflow exports discharge at the right locations and Delft-FIAT uses the same ground elevation as SFINCS. Note that you can also skip these steps and use your own models instead.

First, we build a **SFINCS** model for the user defined region using. 
 - setting from the hydromt_sfincs_config, see the [HydroMT-SFINCS docs](https://deltares.github.io/hydromt_sfincs/latest/) for more info.
 - data from the catalog_path, see the [HydroMT docs](https://deltares.github.io/hydromt/v0.10.0/user_guide/data_prepare_cat.html) for more info.
 - Note that we need src points at the boundary of the SFINCS model (``src_points_output=True``) to get Wflow output at the right locations. Make sure the hydromt_sfincs configuration defines these source locations.


In [ ]:
# Build a SFINCS model
sfincs_build = sfincs.SfincsBuild(
    region=wf.get_ref("$config.region"),
    config=wf.get_ref("$config.hydromt_sfincs_config"),
    sfincs_root="models/sfincs",
    catalog_path=wf.get_ref("$config.catalog_path"),
    plot_fig=wf.get_ref("$config.plot_fig"),
    subgrid_output=True,
    src_points_output=True,
)
wf.create_rule(sfincs_build, rule_id="sfincs_build")

Next we build a **Wflow** model using:
- the sfincs_build output for the model region
- "gauges" based on SFINCS source points
- settings from the hydromt_wflow_config, see [HydroMT-Wflow docs](https://deltares.github.io/hydromt_wflow/latest/) 
- data from the data catalog

In [ ]:
# Build a Wflow model
wflow_build = wflow.WflowBuild(
    region=sfincs_build.output.sfincs_region,
    config=wf.get_ref("$config.hydromt_wflow_config"),
    wflow_root="models/wflow",
    catalog_path=wf.get_ref("$config.catalog_path"),
    gauges=sfincs_build.output.sfincs_src_points,
    plot_fig=wf.get_ref("$config.plot_fig"),
)
wf.create_rule(wflow_build, rule_id="wflow_build")


Next, we build a **FIAT** model using:
- the sfincs_build output for the model region and ground elevation
- settings from the hydromt_fiat_config, see the [hydromt_fiat docs](https://deltares.github.io/hydromt_fiat/latest/)
- data from the data catalog

In [ ]:
# Build a FIAT model
fiat_build = fiat.FIATBuild(
    region=sfincs_build.output.sfincs_region,
    ground_elevation=sfincs_build.output.sfincs_subgrid_dep,
    fiat_root="models/fiat",
    catalog_path=wf.get_ref("$config.catalog_path"),
    config=wf.get_ref("$config.hydromt_fiat_config"),
)
wf.create_rule(fiat_build, rule_id="fiat_build")

### Derive fluvial design events from Wflow

In this section we derive fluvial design events for each Wflow output gauge location.
First, we update and run Wflow to simulate discharge for the present climate. 
Then, we derive design hydrographs for a number of return periods from discharge time series. By default, the magnitude and shape of these events are based on the annual maxima peaks from the timeseries. 

Note that in case of multiple discharge boundary locations this approach assumes full dependence which might be an oversimplification of the reality.

In [ ]:
# Update Wflow meteorological forcing for the simulation period
wflow_update = wflow.WflowUpdateForcing(
    wflow_toml=wflow_build.output.wflow_toml,
    catalog_path=wf.get_ref("$config.catalog_path"),
    start_time=wf.get_ref("$config.start_date"),
    end_time=wf.get_ref("$config.end_date"),
    output_dir=wflow_build.output.wflow_toml.parent/"simulations"/"default"
)
wf.create_rule(wflow_update, rule_id="wflow_update")

In [ ]:
# Run the wflow model for a continuous simulation setup in the wflow_update rule
wflow_run = wflow.WflowRun(
    wflow_toml=wflow_update.output.wflow_out_toml,
    wflow_bin=wf.get_ref("$config.wflow_exe"),
    run_method="exe",  # alternatively use "docker" or "julia"
)
wf.create_rule(wflow_run, rule_id="wflow_run")

In [ ]:
# Derive fluvial design events
# Checkout the FluvialDesignEvents parameters for many options
fluvial_events = discharge.FluvialDesignEvents(
    discharge_nc=wflow_run.output.wflow_output_timeseries,
    rps=wf.get_ref("$config.rps"),
    event_root="input/events",
    index_dim="Q_gauges_bounds",
    wildcard="events",
)

# Note that a new wildcard is created for the fluvial events
wf.create_rule(fluvial_events, rule_id="fluvial_events")

### Derive flood hazard

To derive flood hazard maps for each event, we 
1. Update the SFINCS model using the discharge event timeseries. This will create new SFINCS instances for each event.
2. Run the SFINCS model. This will create simulated water levels for each event.
3. Postprocess the SFINCS output. This will postprocess the SFINCS results to a regular grid of maximum water levels.
4. Optionally, downscale the SFINCS output. This will downscale the max simulated SFINCS water levels to a high-res flood depth map.

In [ ]:
# Update the SFINCS model with fluvial events
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_yaml=fluvial_events.output.event_yaml,
    output_dir=sfincs_build.output.sfincs_inp.parent/"simulations"/"{events}"
)
wf.create_rule(sfincs_update, rule_id="sfincs_update")

In [ ]:
# Run the SFINCS model for each fluvial event
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    sfincs_exe=wf.get_ref("$config.sfincs_exe"),
)
wf.create_rule(sfincs_run, rule_id="sfincs_run")

In [ ]:
# Postprocesses SFINCS results to a regular grid of maximum water levels
sfincs_post = sfincs.SfincsPostprocess(
    sfincs_map=sfincs_run.output.sfincs_map,
)
wf.create_rule(sfincs_post, rule_id="sfincs_post")

In [ ]:
# Optionally, downscale the SFINCS output to derive high-res flood hazard maps
sfincs_downscale = sfincs.SfincsDownscale(
    sfincs_map=sfincs_run.output.sfincs_map,
    sfincs_subgrid_dep=sfincs_build.output.sfincs_subgrid_dep,
    depth_min=wf.get_ref("$config.depth_min"),
    output_root="output/hazard",
)
wf.create_rule(sfincs_downscale, rule_id="sfincs_downscale")

## Derive flood risk

To calculate flood risk, we 
- Update Delft-FIAT with *all fluvial events* which are combined in an event set. This will create a new Delft-FIAT instance for the event set.
- Run Delft-FIAT to calculate flood impact and risk. This will create impact and risk data at the individual and aggregated asset level.
- Visualize the risk results at the aggregated asset level.


In [ ]:
# Update FIAT hazard forcing with the fluvial eventset to compute fluvial flood risk
fiat_update = fiat.FIATUpdateHazard(
    fiat_cfg=fiat_build.output.fiat_cfg,
    event_set_yaml=fluvial_events.output.event_set_yaml,
    map_type="water_level",
    hazard_maps=sfincs_post.output.sfincs_zsmax,
    risk=True,
    output_dir=fiat_build.output.fiat_cfg.parent/"simulations"
)
wf.create_rule(fiat_update, rule_id="fiat_update")

In [ ]:
# Run FIAT to compute pluvial flood risk
fiat_run = fiat.FIATRun(
    fiat_cfg=fiat_update.output.fiat_out_cfg,
    fiat_exe=wf.get_ref("$config.fiat_exe"),
)
wf.create_rule(fiat_run, rule_id="fiat_run")

In [ ]:
# Visualize Fiat 
fiat_visualize = fiat.FIATVisualize(
    fiat_cfg=fiat_update.output.fiat_out_cfg,
    fiat_output_csv=fiat_run.output.fiat_out_csv,
    spatial_joins_cfg=fiat_build.output.spatial_joins_cfg,
    output_dir="output/risk"
)
wf.create_rule(fiat_visualize, rule_id="fiat_visualize")

## Visualize and execute the workflow

To inspect the workflow we can plot the rulegraph which shows all rules their dependencies.
The nodes are colored based on the type, for instance the red nodes show the result rules.

In [ ]:
# plot the rulegraph using graphviz
wf.plot_rulegraph(filename="rulegraph.svg", plot_rule_attrs=True)

In [ ]:
# dryrun workflow. Make sure no warnings are raised
wf.dryrun()

The workflow can be executed using HydroFlows or a workflow engine. 
To run the workflow in HydroFlows use ``wf.run()``. 
To run the workflow with SnakeMake (preferred) use ``wf.to_snakemake()`` to create a snakemake file, see below.
You can then use the Snakemake CLI to execute the workflow, see the [snakemake documentation](https://snakemake.readthedocs.io/en/stable/executing/cli.html)

In [ ]:
# Write the workflow to a Snakefile and snakefile.config.yml
wf.to_snakemake()

# show the files in the case directory
print(f"{wf.root.relative_to(pwd)}:")
for f in wf.root.iterdir():
    print(f"- {f.name}")

In [ ]:
# uncomment to run the workflow
# import subprocess
# subprocess.run(["snakemake", "-c", "1"], cwd=wf.root)